In [ ]:
import openai
from openai import OpenAI
client = OpenAI()

In [6]:
response  = client.chat.completions.create(
    model = 'gpt-4',
    messages = [{"role": "user", "content":"is it too late to join the course?"}]
)

response.choices[0].message.content 

"As an AI, I don't have specific information about a course's schedule. Please contact the course administrator or institution directly for accurate information."